## Инициализация библиотек

In [ ]:
# Основные библиотеки
!pip install pybind11
!pip install pandas openpyxl numpy
# Для возможности использовать transformers (опционально)
!pip install transformers torch

# Для дополнительных возможностей
!pip install scikit-learn 
!pip install gensim 

# Библиотеки для русского языка
!pip install razdel
!pip install pymorphy3 chardet

# Для Dostoevsky нужна модель
!pip install dostoevsky --no-deps
!pip install fasttext-wheel==0.9.2

#!python -m dostoevsky download fasttext-social-network-model


## Импорт и проверка достаточности библиотек и моделей

In [24]:
"""
Анализатор упоминаний для мониторинга репутации брендов
Использует специализированные библиотеки для русского языка
"""

import re
import pandas as pd
import numpy as np
import os
import sys
import glob
import warnings
from typing import List, Dict, Optional, Tuple
import chardet
warnings.filterwarnings('ignore')

# Проверка и установка недостающих библиотек
def install_missing_packages():
    """Установка недостающих пакетов"""
    required_packages = {
        'pandas': 'pandas',
        'numpy': 'numpy',
        'openpyxl': 'openpyxl',
        'xlrd': 'xlrd>=1.2.0',
        'chardet': 'chardet',
        'tqdm': 'tqdm'
    }
    
    missing = []
    for package, install_name in required_packages.items():
        try:
            __import__(package)
        except ImportError:
            missing.append(install_name)
    
    if missing:
        print(f"Обнаружены отсутствующие пакеты: {', '.join(missing)}")
        response = input("Установить их автоматически? (y/n): ")
        if response.lower() == 'y':
            import subprocess
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install"] + missing)
                print("Пакеты успешно установлены!")
            except Exception as e:
                print(f"Ошибка при установке пакетов: {e}")
                print("Пожалуйста, установите их вручную:")
                print(f"pip install {' '.join(missing)}")
        else:
            print("Продолжаем без установки...")

# Вызываем проверку пакетов
install_missing_packages()

print("="*70)
print("АВТОМАТИЧЕСКИЙ АНАЛИЗАТОР УПОМЕНАНИЙ ДЛЯ НЕСКОЛЬКИХ БРЕНДОВ")
print("="*70)

# Конфигурации для разных брендов
BRAND_CONFIGS = {
    1: {
        'name': 'Газпром нефть',
        'keywords': [
            'газпромнефть', 'газпром', 'азс газпром', 
            'гпн', 'gazprom', 'gazpromneft', 'gdrive',
            'газпром нефть', 'газпромнефти', 'газпромнефтью'
        ],
        'risk_words': [
            'бадяж', 'некачествен', 'плох', 'ужас', 'кошмар',
            'обман', 'развод', 'жульнич', 'брак', 'испорч',
            'грязн', 'вредн', 'опасн', 'трясет', 'дергает',
            'дым', 'троит', 'глохнет', 'яма', 'разбит'
        ],
        'positive_words': [
            'хорош', 'отличн', 'прекрасн', 'замечательн',
            'рекомендую', 'советую', 'спасибо', 'благодарю',
            'доволен', 'рад', 'класс', 'супер', 'лучш'
        ],
        'competitors': [
            'лукойл', 'роснефть', 'татнефть', 'shell', 'бп',
            'energy', 'энергия', 'аи', 'нефтемагистраль'
        ],
        'output_prefix': 'ГПН'
    },
    2: {
        'name': 'Малютка',
        'keywords': [
            'малютка', 'nutricia', 'нутриция', 'нутриция',
            'малютк', 'малютки', 'малютку', 'малюткой',
            'malyutka', 'детск', 'питани', 'смес', 'каш',
            'молочк', 'пюре'
        ],
        'risk_words': [
            'запор', 'колики', 'сыпь', 'рвота', 'аллерги',
            'плох', 'ужас', 'кошмар', 'некачествен', 'вредн',
            'опасн', 'отравлен', 'болезн', 'больн', 'лечен'
        ],
        'positive_words': [
            'вкусн', 'нравится', 'люблю', 'обожаю', 'отличн',
            'хорош', 'рекомендую', 'спасибо', 'доверяю',
            'полезн', 'здоров', 'качествен', 'натуральн'
        ],
        'competitors': [
            'нутрилак', 'нестле', 'хипп', 'семпер', 'фрисо',
            'агуша', 'тема', 'беллакт', 'беби'
        ],
        'output_prefix': 'Малютка'
    }
}



class ModelDownloader:
    """Класс для скачивания и управления моделями"""
    
    @staticmethod
    def download_dostoevsky_model():
        """Скачивание модели Dostoevsky"""
        print("Попытка скачать модель Dostoevsky...")
        try:
            import requests
            import zipfile
            import tempfile
            import shutil
            
            # URL модели Dostoevsky
            model_url = "https://github.com/bureaucratic-labs/dostoevsky/raw/master/dostoevsky/data/models/fasttext-social-network-model.bin"
            
            # Создаем временную директорию
            temp_dir = tempfile.mkdtemp()
            model_path = os.path.join(temp_dir, "fasttext-social-network-model.bin")
            
            # Скачиваем модель
            print(f"Скачивание модели Dostoevsky...")
            response = requests.get(model_url, stream=True)
            total_size = int(response.headers.get('content-length', 0))
            
            if response.status_code == 200:
                with open(model_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                
                # Копируем в нужную директорию
                import dostoevsky
                dostoevsky_dir = os.path.dirname(dostoevsky.__file__)
                target_dir = os.path.join(dostoevsky_dir, 'data', 'models')
                os.makedirs(target_dir, exist_ok=True)
                target_path = os.path.join(target_dir, 'fasttext-social-network-model.bin')
                
                shutil.copy(model_path, target_path)
                print(f"Модель Dostoevsky успешно скачана в: {target_path}")
                return True
            else:
                print(f"Ошибка скачивания модели: {response.status_code}")
                return False
                
        except Exception as e:
            print(f"Ошибка при скачивании модели Dostoevsky: {e}")
            print("Используем альтернативную модель...")
            return False
    
    @staticmethod
    def check_and_fix_dostoevsky():
        """Проверка и исправление Dostoevsky"""
        try:
            from dostoevsky.tokenization import RegexTokenizer
            from dostoevsky.models import FastTextSocialNetworkModel
            
            # Пробуем загрузить модель
            tokenizer = RegexTokenizer()
            model = FastTextSocialNetworkModel(tokenizer=tokenizer)
            
            # Проверяем работу на тестовом тексте
            test_result = model.predict(["тестовый текст"], k=2)
            
            print("✓ Dostoevsky проверен и работает корректно")
            return True
            
        except Exception as e:
            print(f"Dostoevsky требует настройки: {str(e)[:100]}")
            
            # Пробуем скачать модель
            return ModelDownloader.download_dostoevsky_model()

#ModelDownloader()

АВТОМАТИЧЕСКИЙ АНАЛИЗАТОР УПОМЕНАНИЙ ДЛЯ НЕСКОЛЬКИХ БРЕНДОВ


## Улучшенный анализатор тональности с автовыбором моделей

In [25]:
class SentimentAnalyzer:
    """Анализатор тональности"""
    
    def __init__(self):
        self.model = None
        self._init_model()
    
    def _init_model(self):
        """Инициализация модели анализа тональности"""
        print("  Инициализация анализатора тональности...")
        
        # Пробуем загрузить Dostoevsky
        try:
            from dostoevsky.tokenization import RegexTokenizer
            from dostoevsky.models import FastTextSocialNetworkModel
            
            self.tokenizer = RegexTokenizer()
            self.model = FastTextSocialNetworkModel(tokenizer=self.tokenizer)
            print("  ✓ Dostoevsky загружен")
            self.model_type = 'dostoevsky'
            
        except Exception as e:
            print(f"  ✗ Dostoevsky не загрузился: {e}")
            print("  Используем базовый анализатор по ключевым словам")
            self.model_type = 'basic'
    
    def analyze(self, text: str, risk_words: List[str], positive_words: List[str]) -> Dict:
        """
        Анализ тональности текста
        
        Args:
            text: Текст для анализа
            risk_words: Слова, указывающие на риск
            positive_words: Позитивные маркеры
            
        Returns:
            Dict: Результаты анализа
        """
        if not text or len(text.strip()) < 3:
            return {'sentiment': 'Нейтральная', 'confidence': 0.5}
        
        text_lower = text.lower()
        
        # Если есть модель Dostoevsky, используем её
        if self.model_type == 'dostoevsky' and self.model:
            try:
                results = self.model.predict([text], k=5)[0]
                
                sentiment_map = {
                    'positive': 'Позитивная',
                    'negative': 'Негативная', 
                    'neutral': 'Нейтральная'
                }
                
                if results:
                    main_sentiment = max(results.items(), key=lambda x: x[1])
                    sentiment_label = sentiment_map.get(main_sentiment[0], 'Нейтральная')
                    
                    return {
                        'sentiment': sentiment_label,
                        'confidence': float(main_sentiment[1]),
                        'model': 'dostoevsky'
                    }
            except Exception as e:
                print(f"    Ошибка Dostoevsky: {e}")
        
        # Базовый анализ по ключевым словам
        return self._basic_analysis(text_lower, risk_words, positive_words)
    
    def _basic_analysis(self, text_lower: str, risk_words: List[str], positive_words: List[str]) -> Dict:
        """Базовый анализ тональности по ключевым словам"""
        # Подсчет риск-слов
        risk_count = sum(1 for word in risk_words if word in text_lower)
        
        # Подсчет позитивных слов
        positive_count = sum(1 for word in positive_words if word in text_lower)
        
        # Эмоциональные маркеры
        negative_emojis = ['😠', '😡', '🤬', '😢', '😭', '😤', '👎', '💔']
        positive_emojis = ['😀', '😃', '😄', '😁', '😆', '😍', '🤩', '👍', '❤️']
        
        for emoji in negative_emojis:
            if emoji in text_lower:
                risk_count += 1
        
        for emoji in positive_emojis:
            if emoji in text_lower:
                positive_count += 1
        
        # Ключевые паттерны для русского языка
        negative_patterns = [
            r'\b(плох|ужасн|кошмар|отвратительн|некачествен|жалоб|недоволен)\b',
            r'\b(возмущен|разочарован|протестую|требую|верните)\b'
        ]
        
        positive_patterns = [
            r'\b(отличн|прекрасн|замечательн|супер|класс|лучш|хорош)\b',
            r'\b(рекомендую|советую|спасибо|благодарю|доволен|рад)\b'
        ]
        
        for pattern in negative_patterns:
            if re.search(pattern, text_lower):
                risk_count += 1
        
        for pattern in positive_patterns:
            if re.search(pattern, text_lower):
                positive_count += 1
        
        # Определение тональности
        if risk_count > positive_count and risk_count > 0:
            confidence = min(0.5 + (risk_count - positive_count) * 0.1, 0.9)
            return {'sentiment': 'Негативная', 'confidence': confidence, 'model': 'basic'}
        elif positive_count > risk_count and positive_count > 0:
            confidence = min(0.5 + (positive_count - risk_count) * 0.1, 0.9)
            return {'sentiment': 'Позитивная', 'confidence': confidence, 'model': 'basic'}
        elif risk_count == positive_count and risk_count > 0:
            return {'sentiment': 'Смешанная', 'confidence': 0.5, 'model': 'basic'}
        else:
            return {'sentiment': 'Нейтральная', 'confidence': 0.5, 'model': 'basic'}


## Основной анализатор упоминаний

In [26]:
class MentionAnalyzer:
    """Анализатор упоминаний для конкретного бренда"""
    
    # Общие теги для всех брендов
    UNIVERSAL_TAGS = [
        'Качество продукта/услуги',
        'Сравнение с конкурентами', 
        'Жалоба/проблема',
        'Положительный отзыв',
        'Вопрос/справка',
        'Рекомендация/совет',
        'Цена/стоимость',
        'Сервис/обслуживание',
        'Технические вопросы',
        'Безопасность/здоровье'
    ]
    
    def __init__(self, config: Dict):
        """
        Args:
            config: Конфигурация бренда
        """
        self.config = config
        self.brand_name = config['name']
        self.keywords = config['keywords']
        self.risk_words = config['risk_words']
        self.positive_words = config['positive_words']
        self.competitors = config['competitors']
        
        # Инициализация анализатора тональности
        self.sentiment_analyzer = SentimentAnalyzer()
        
        # Статистика
        self.stats = {
            'total': 0,
            'relevant': 0,
            'negative': 0,
            'positive': 0,
            'dangerous': 0
        }
        
        print(f"  Анализатор инициализирован для: {self.brand_name}")
    
    def is_relevant(self, text: str) -> bool:
        """Проверка релевантности текста"""
        if not text or len(text.strip()) < 5:
            return False
        
        text_lower = text.lower()
        
        # Проверка ключевых слов
        for keyword in self.keywords:
            if keyword in text_lower:
                return True
        
        # Проверка частичных совпадений
        words = re.findall(r'\b\w+\b', text_lower)
        for word in words:
            for keyword in self.keywords:
                if keyword in word:
                    return True
        
        return False
    
    def extract_tags(self, text: str) -> List[str]:
        """Извлечение тегов из текста"""
        tags = []
        text_lower = text.lower()
        
        # 1. Качество продукта
        quality_patterns = [r'качеств', r'некачествен', r'плох', r'ужас', r'кошмар', r'бадяж']
        if any(re.search(pattern, text_lower) for pattern in quality_patterns):
            tags.append('Качество продукта/услуги')
        
        # 2. Жалобы
        if re.search(r'жалоб|недовол|возмущ|протест|претенз', text_lower):
            tags.append('Жалоба/проблема')
        
        # 3. Позитивные отзывы
        if re.search(r'отличн|прекрасн|замечательн|хорош|рекомендую|спасибо', text_lower):
            tags.append('Положительный отзыв')
        
        # 4. Сравнение с конкурентами
        if self.competitors and any(comp in text_lower for comp in self.competitors):
            tags.append('Сравнение с конкурентами')
        
        # 5. Вопросы
        if '?' in text or any(word in text_lower for word in ['как', 'что', 'где', 'подскажите']):
            tags.append('Вопрос/справка')
        
        # 6. Цена
        if re.search(r'цен|стоимост|дорог|дешев', text_lower):
            tags.append('Цена/стоимость')
        
        # 7. Сервис
        if re.search(r'сервис|обслуживан|персонал|консультац', text_lower):
            tags.append('Сервис/обслуживание')
        
        # 8. Рекомендации
        if re.search(r'рекоменд|совет', text_lower):
            tags.append('Рекомендация/совет')
        
        # Убираем дубликаты и ограничиваем 5 тегами
        unique_tags = []
        for tag in tags:
            if tag not in unique_tags:
                unique_tags.append(tag)
        
        return unique_tags[:5]
    
    def analyze_text(self, text: str) -> Dict:
        """
        Полный анализ одного текста
        
        Returns:
            Dict: Результаты анализа
        """
        self.stats['total'] += 1
        
        # Проверка релевантности
        relevant = self.is_relevant(text)
        if not relevant:
            return {
                'relevant': False,
                'sentiment': 'Нерелевантно',
                'tags': [],
                'dangerous': False
            }
        
        self.stats['relevant'] += 1
        
        # Анализ тональности
        sentiment_result = self.sentiment_analyzer.analyze(text, self.risk_words, self.positive_words)
        sentiment = sentiment_result['sentiment']
        
        # Обновляем статистику тональности
        if sentiment == 'Негативная':
            self.stats['negative'] += 1
        elif sentiment == 'Позитивная':
            self.stats['positive'] += 1
        
        # Извлечение тегов
        tags = self.extract_tags(text)
        
        # Оценка опасности
        dangerous = self._is_dangerous(text, sentiment, tags)
        if dangerous:
            self.stats['dangerous'] += 1
        
        return {
            'relevant': True,
            'sentiment': sentiment,
            'confidence': sentiment_result.get('confidence', 0.5),
            'tags': tags,
            'dangerous': dangerous,
            'model': sentiment_result.get('model', 'basic')
        }
    
    def _is_dangerous(self, text: str, sentiment: str, tags: List[str]) -> bool:
        """Определение опасности для репутации"""
        text_lower = text.lower()
        danger_score = 0
        
        # 1. Негативная тональность
        if sentiment == 'Негативная':
            danger_score += 2
        
        # 2. Жалоба в тегах
        if 'Жалоба/проблема' in tags:
            danger_score += 2
        
        # 3. Риск-слова
        for risk_word in self.risk_words:
            if risk_word in text_lower:
                danger_score += 1
        
        # 4. Призывы к действию
        if re.search(r'не\s+покупай|не\s+пользуй|бойкотир|распространи|предупреди', text_lower):
            danger_score += 2
        
        # Опасным считается сообщение с 3+ баллами
        return danger_score >= 3
    
    def print_stats(self):
        """Вывод статистики анализа"""
        print(f"\n  Статистика для {self.brand_name}:")
        print(f"    Всего текстов: {self.stats['total']}")
        
        if self.stats['total'] > 0:
            relevant_pct = (self.stats['relevant'] / self.stats['total']) * 100
            print(f"    Релевантных: {self.stats['relevant']} ({relevant_pct:.1f}%)")
            
            if self.stats['relevant'] > 0:
                negative_pct = (self.stats['negative'] / self.stats['relevant']) * 100
                positive_pct = (self.stats['positive'] / self.stats['relevant']) * 100
                
                print(f"    Негативных: {self.stats['negative']} ({negative_pct:.1f}%)")
                print(f"    Позитивных: {self.stats['positive']} ({positive_pct:.1f}%)")
                print(f"    Опасных для репутации: {self.stats['dangerous']}")

## Обработчик файлов

In [27]:
class FileManager:
    """Управление файлами с нумерацией"""
    
    @staticmethod
    def find_input_files() -> Dict[int, str]:
        """
        Поиск входных файлов по шаблону date_reqN.расширение
        
        Returns:
            Dict[int, str]: Словарь {номер: путь_к_файлу}
        """
        files_found = {}
        
        # Шаблоны для поиска
        patterns = [
            'date_req[0-9]*.xls',
            'date_req[0-9]*.xlsx', 
            'date_req[0-9]*.csv',
            'date_req[0-9]*.txt'
        ]
        
        print("\nПоиск входных файлов...")
        
        for pattern in patterns:
            for file_path in glob.glob(pattern):
                # Извлекаем номер из имени файла
                match = re.search(r'date_req(\d+)', file_path, re.IGNORECASE)
                if match:
                    file_num = int(match.group(1))
                    if file_num in BRAND_CONFIGS:
                        files_found[file_num] = file_path
                        print(f"  ✓ Найден файл для {BRAND_CONFIGS[file_num]['name']}: {file_path}")
                    else:
                        print(f"  ⚠ Файл {file_path} имеет номер {file_num}, но конфигурация для этого номера не определена")
                else:
                    # Файл без номера (просто date_req)
                    print(f"  ⚠ Файл без номера: {file_path}")
        
        # Также проверяем файлы с явными названиями
        specific_files = {
            'date_req_gpn.xls': 1,
            'date_req_gpn.xlsx': 1,
            'date_req_malyutka.xls': 2,
            'date_req_malyutka.xlsx': 2,
            'date_req_gazprom.xls': 1,
            'date_req_gazprom.xlsx': 1
        }
        
        for file_name, file_num in specific_files.items():
            if os.path.exists(file_name) and file_num not in files_found:
                files_found[file_num] = file_name
                print(f"  ✓ Найден файл по имени для {BRAND_CONFIGS[file_num]['name']}: {file_name}")
        
        if not files_found:
            print("  ⚠ Входные файлы не найдены!")
            print("  Ожидаются файлы с именами:")
            print("    - date_req1.xls/date_req1.xlsx - для Газпром нефть")
            print("    - date_req2.xls/date_req2.xlsx - для Малютка")
            print("    - date_req_gpn.xls - для Газпром нефть")
            print("    - date_req_malyutka.xls - для Малютка")
        
        return files_found
    
    @staticmethod
    def generate_output_filename(file_num: int, input_path: str) -> str:
        """
        Генерация имени выходного файла
        
        Args:
            file_num: Номер файла (1 или 2)
            input_path: Путь к входному файлу
            
        Returns:
            str: Имя выходного файла
        """
        brand_name = BRAND_CONFIGS[file_num]['output_prefix']
        input_name = os.path.basename(input_path)
        
        # Определяем расширение
        if input_path.endswith('.xlsx'):
            output_ext = '.xlsx'
        elif input_path.endswith('.xls'):
            output_ext = '.xlsx'  # Сохраняем в современном формате
        elif input_path.endswith('.csv'):
            output_ext = '.csv'
        else:
            output_ext = '.xlsx'
        
        # Генерируем имя файла
        timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
        
        # Варианты имен:
        # 1. С номером: date_res1.xlsx
        # 2. С префиксом бренда: ГПН_результат_20241215.xlsx
        # 3. Комбинированное: date_res1_ГПН.xlsx
        
        output_names = [
            f"date_res{file_num}{output_ext}",  # По номеру
            f"{brand_name}_результат_{timestamp}{output_ext}",  # С брендом и временем
            f"date_res{file_num}_{brand_name}{output_ext}",  # Комбинированное
            f"результат_{brand_name}{output_ext}"  # Простое
        ]
        
        # Выбираем первое свободное имя
        for name in output_names:
            if not os.path.exists(name):
                return name
        
        # Если все имена заняты, добавляем номер
        counter = 1
        while True:
            name = f"date_res{file_num}_{brand_name}_{counter}{output_ext}"
            if not os.path.exists(name):
                return name
            counter += 1
    
    @staticmethod
    def read_file(file_path: str) -> pd.DataFrame:
        """
        Чтение файла с автоматическим определением формата
        
        Args:
            file_path: Путь к файлу
            
        Returns:
            pd.DataFrame: Прочитанные данные
        """
        print(f"  Чтение файла: {file_path}")
        
        try:
            if file_path.endswith('.xlsx'):
                df = pd.read_excel(file_path, engine='openpyxl')
            elif file_path.endswith('.xls'):
                # Пробуем разные движки для старых .xls файлов
                try:
                    df = pd.read_excel(file_path, engine='xlrd')
                except:
                    df = pd.read_excel(file_path, engine='openpyxl')
            elif file_path.endswith('.csv'):
                # Пробуем разные кодировки для CSV
                encodings = ['utf-8', 'cp1251', 'windows-1251', 'latin1']
                for encoding in encodings:
                    try:
                        df = pd.read_csv(file_path, encoding=encoding, sep=None, engine='python')
                        print(f"    Прочитано с кодировкой: {encoding}")
                        break
                    except:
                        continue
                else:
                    raise ValueError("Не удалось прочитать CSV файл")
            else:
                raise ValueError(f"Неподдерживаемый формат файла: {file_path}")
            
            print(f"  ✓ Успешно прочитано {len(df)} строк")
            
            # Ищем колонку с текстом
            text_columns = [col for col in df.columns if any(word in col.lower() 
                           for word in ['текст', 'text', 'сообщение', 'message', 'комментарий'])]
            
            if text_columns:
                if 'Текст' not in df.columns and text_columns[0] != 'Текст':
                    df = df.rename(columns={text_columns[0]: 'Текст'})
                    print(f"  ✓ Переименована колонка '{text_columns[0]}' в 'Текст'")
            elif 'Текст' not in df.columns and len(df.columns) > 0:
                # Если нет явной колонки с текстом, используем первую колонку
                first_col = df.columns[0]
                df = df.rename(columns={first_col: 'Текст'})
                print(f"  ✓ Используем первую колонку '{first_col}' как 'Текст'")
            
            return df
            
        except Exception as e:
            print(f"  ✗ Ошибка чтения файла: {e}")
            # Создаем пустой DataFrame с колонкой Текст
            return pd.DataFrame(columns=['Текст'])

## Получение конфигурации для бренда

In [28]:
class BrandAnalyzer:
    """Основной класс для анализа брендов"""
    
    def __init__(self):
        self.file_manager = FileManager()
        self.processed_files = []
    
    def process_brand(self, file_num: int, input_path: str) -> Optional[str]:
        """
        Обработка одного бренда
        
        Args:
            file_num: Номер файла (1 или 2)
            input_path: Путь к входному файлу
            
        Returns:
            Optional[str]: Путь к выходному файлу или None в случае ошибки
        """
        print(f"\n{'='*60}")
        print(f"Обработка файла {file_num}: {BRAND_CONFIGS[file_num]['name']}")
        print(f"{'='*60}")
        
        # Загрузка конфигурации
        config = BRAND_CONFIGS[file_num]
        
        # Чтение файла
        df = self.file_manager.read_file(input_path)
        
        if df.empty or 'Текст' not in df.columns:
            print(f"  ✗ Файл не содержит данных или колонки 'Текст'")
            return None
        
        # Создание анализатора
        analyzer = MentionAnalyzer(config)
        
        # Анализ каждого текста
        print(f"  Начало анализа {len(df)} строк...")
        
        results = []
        for idx, row in df.iterrows():
            text = str(row['Текст']) if pd.notna(row['Текст']) else ''
            
            # Анализ текста
            analysis = analyzer.analyze_text(text)
            
            # Формирование результата
            result = {
                'Сообщение релевантно?': 'Да' if analysis['relevant'] else 'Нет',
                'Тональность': analysis['sentiment'],
                'Опасно для репутации': 'Да' if analysis['dangerous'] else 'Нет',
                'Модель анализа': analysis.get('model', 'basic'),
                'Уверенность': f"{analysis.get('confidence', 0.5):.2f}"
            }
            
            # Добавляем теги
            tags = analysis['tags']
            for i in range(1, 6):
                result[f'Тег {i}'] = tags[i-1] if i <= len(tags) else ''
            
            results.append(result)
            
            # Прогресс
            if (idx + 1) % 50 == 0:
                print(f"    Обработано {idx + 1}/{len(df)} строк...")
        
        # Создание DataFrame с результатами
        results_df = pd.DataFrame(results)
        
        # Объединение с исходными данными
        output_df = pd.concat([df, results_df], axis=1)
        
        # Генерация имени выходного файла
        output_path = self.file_manager.generate_output_filename(file_num, input_path)
        
        # Сохранение результатов
        try:
            if output_path.endswith('.xlsx'):
                output_df.to_excel(output_path, index=False, engine='openpyxl')
            elif output_path.endswith('.csv'):
                output_df.to_csv(output_path, index=False, encoding='utf-8-sig')
            else:
                output_path = output_path + '.xlsx'
                output_df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"  ✓ Результаты сохранены в: {output_path}")
            
            # Вывод статистики
            analyzer.print_stats()
            
            self.processed_files.append({
                'brand': config['name'],
                'input': input_path,
                'output': output_path,
                'stats': analyzer.stats
            })
            
            return output_path
            
        except Exception as e:
            print(f"  ✗ Ошибка сохранения файла: {e}")
            return None
    
    def process_all_files(self):
        """Обработка всех найденных файлов"""
        # Поиск файлов
        files = self.file_manager.find_input_files()
        
        if not files:
            print("\n⚠ Не найдено файлов для обработки!")
            print("  Создаем тестовые файлы для демонстрации...")
            self._create_test_files()
            files = self.file_manager.find_input_files()
        
        # Сортировка по номеру файла
        sorted_files = sorted(files.items(), key=lambda x: x[0])
        
        # Обработка каждого файла
        results = []
        for file_num, file_path in sorted_files:
            if file_num in BRAND_CONFIGS:
                result_path = self.process_brand(file_num, file_path)
                if result_path:
                    results.append((file_num, result_path))
            else:
                print(f"\n⚠ Пропускаем файл {file_path}: номер {file_num} не поддерживается")
        
        # Вывод итогов
        self._print_summary(results)
    
    def _create_test_files(self):
        """Создание тестовых файлов если нет входных"""
        test_data_gpn = [
            "АЗС Газпром на Щербакова 2. Видимо опять начали бадяжить бензин... Две недели назад залили полный бак 95го.",
            "Ребята нужна помощь, человек остался без денег, стоит на заправке Газпромнефть на улице Ленина 129.",
            "У тебя даже на крышке бака написано, что минимум 95. Я в свой фф3 лью АИ-100 на газпроме, очень хорошо машине, расход 8.3",
            "смех, газпромнефть сам себя проверяет на вшивость) это как коррумпированные люди в отделе борьбы с коррупцией)",
            "Хорошее топливо. Хотя качество бензина жалуются.",
        ]
        
        test_data_malyutka = [
            "С 3 недель подкармливаю смесью, начали с малютки, начались запоры. Нас посоветовали нутрилак.",
            "Мой сын просто обожает детское молочко Малютка! Выпивает по целой бутылочке перед сном.",
            "Малютка очень вкусные каши, мои обе ели хорошо именно гречневую. Они прямо пахнут вкусно.",
            "Кристина: малютка самая плохая из всех смесей что есть на прилавках России, по составу.",
            "Расскажите пожалуйста про смесь «малютка» от 0, преимущества и недостатки?",
        ]
        
        # Сохраняем тестовые данные
        pd.DataFrame({'Текст': test_data_gpn}).to_excel('date_req1.xlsx', index=False)
        pd.DataFrame({'Текст': test_data_malyutka}).to_excel('date_req2.xlsx', index=False)
        
        print("  ✓ Созданы тестовые файлы: date_req1.xlsx и date_req2.xlsx")
    
    def _print_summary(self, results: List[Tuple[int, str]]):
        """Вывод итогового отчета"""
        print(f"\n{'='*70}")
        print("ИТОГОВЫЙ ОТЧЕТ")
        print("="*70)
        
        if not results:
            print("  Не обработано ни одного файла")
            return
        
        print(f"  Обработано файлов: {len(results)}")
        print()
        
        total_stats = {
            'total_texts': 0,
            'relevant_texts': 0,
            'negative_texts': 0,
            'dangerous_texts': 0
        }
        
        for file_num, output_path in results:
            brand_name = BRAND_CONFIGS[file_num]['name']
            print(f"  {brand_name}:")
            print(f"    Результат сохранен в: {output_path}")
            
            # Находим статистику для этого файла
            for processed in self.processed_files:
                if processed['output'] == output_path:
                    stats = processed['stats']
                    print(f"    Проанализировано текстов: {stats['total']}")
                    print(f"    Релевантных: {stats['relevant']}")
                    print(f"    Негативных: {stats['negative']}")
                    print(f"    Опасных для репутации: {stats['dangerous']}")
                    
                    # Суммируем общую статистику
                    total_stats['total_texts'] += stats['total']
                    total_stats['relevant_texts'] += stats['relevant']
                    total_stats['negative_texts'] += stats['negative']
                    total_stats['dangerous_texts'] += stats['dangerous']
                    break
        
        # Общая статистика
        print(f"\n  ОБЩАЯ СТАТИСТИКА:")
        print(f"    Всего проанализировано текстов: {total_stats['total_texts']}")
        if total_stats['total_texts'] > 0:
            relevant_pct = (total_stats['relevant_texts'] / total_stats['total_texts']) * 100
            print(f"    Релевантных текстов: {total_stats['relevant_texts']} ({relevant_pct:.1f}%)")
            
            if total_stats['relevant_texts'] > 0:
                negative_pct = (total_stats['negative_texts'] / total_stats['relevant_texts']) * 100
                print(f"    Негативных упоминаний: {total_stats['negative_texts']} ({negative_pct:.1f}%)")
                print(f"    Опасных для репутации: {total_stats['dangerous_texts']}")
        
        print("="*70)

## Основная функция

In [30]:
def main():
    """Основная функция"""
    print("\nЗапуск анализа упоминаний для нескольких брендов...")
    print("Ожидаются файлы с номерами:")
    print("  date_req1.xls/xlsx - для Газпром нефть")
    print("  date_req2.xls/xlsx - для Малютка")
    print("  или date_req_gpn.xls, date_req_malyutka.xls")
    print("-"*70)
    
    # Проверка зависимостей
    try:
        import pandas as pd
        print("✓ Pandas доступен")
    except ImportError:
        print("✗ Pandas не установлен. Установите: pip install pandas openpyxl xlrd")
        return
    
    try:
        import dostoevsky
        print("✓ Dostoevsky доступен")
    except ImportError:
        print("⚠ Dostoevsky не установлен. Будет использован базовый анализ.")
        print("  Для улучшения точности установите: pip install dostoevsky fasttext")
    
    # Запуск анализа
    analyzer = BrandAnalyzer()
    analyzer.process_all_files()
    
    print("\nАнализ завершен!")
    print("="*70)


if __name__ == "__main__":
    main()


Запуск анализа упоминаний для нескольких брендов...
Ожидаются файлы с номерами:
  date_req1.xls/xlsx - для Газпром нефть
  date_req2.xls/xlsx - для Малютка
  или date_req_gpn.xls, date_req_malyutka.xls
----------------------------------------------------------------------
✓ Pandas доступен
✓ Dostoevsky доступен

Поиск входных файлов...
  ✓ Найден файл для Газпром нефть: date_req1.xls
  ✓ Найден файл для Малютка: date_req2.xls

Обработка файла 1: Газпром нефть
  Чтение файла: date_req1.xls
  ✓ Успешно прочитано 11 строк
  Инициализация анализатора тональности...
  ✗ Dostoevsky не загрузился: No module named 'fasttext'
  Используем базовый анализатор по ключевым словам
  Анализатор инициализирован для: Газпром нефть
  Начало анализа 11 строк...
  ✓ Результаты сохранены в: ГПН_результат_20260208_134650.xlsx

  Статистика для Газпром нефть:
    Всего текстов: 11
    Релевантных: 10 (90.9%)
    Негативных: 3 (30.0%)
    Позитивных: 2 (20.0%)
    Опасных для репутации: 3

Обработка файла 2: